In [1]:
#https://medium.com/@kenneth.ca95/a-guide-to-transfer-learning-with-keras-using-resnet50-a81a4a28084b
# Running the version as 1.x is optional, without that first line it will run the last version of tensorflow for Colab.

#import keras
import tensorflow as tf 
from tensorflow import keras
import tensorflow.keras as K

from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Activation
from keras.layers import Conv2D, MaxPooling2D

import pandas as pd
#from keras.utils import plot_model
from keras.utils.vis_utils import plot_model
#from keras.utils import np_utils
from keras import utils as np_utils
#from keras.utils import to_categorical
from tensorflow.keras.utils import to_categorical
pd.set_option('display.max_columns',None)#displaying long list of columns
pd.set_option('display.max_rows', None)#displaying long list of rows
pd.set_option('display.width', 1000)#width of window




In [2]:
# Load data
# CIFAR-10 is a dataset with 60000 32x32 colour images grouped in 10 classes, that means 6000 images per class. 
# This is a dataset of 50,000 32x32 color training images and 10,000 test images, labeled over 10 categories.
# The categories are airplane, automobile, beer, cat, deer, dog, frog, horse, ship, truck. 
# We can take advantage of the fact that these categories and a lot more are into the Imagenet collection.
# Loading the CIFAR-10 datasets
from keras.datasets import cifar10


In [3]:
# Preprocess data function
# Now that the data is loaded, we are going to build a preprocess function for the data. 
# We have X as a numpy array of shape (m, 32, 32, 1) where m is the number of images, 
# 28 and 28 the dimensions, and 1 is because we use grayscale images. 
# We have a set of X for training and a set of X for validation. 
# Y is a numpy array of shape (m, ) that we want to be our labels. 
# Since we work with 10 different categories, we make use of one-hot encoding with a 
# function of Keras that makes our Y into a shape of (m, 10). That also applies for the validation.

def preprocess_data(X,Y):
  X_p = K.applications.xception.preprocess_input(X)
  Y_p = K.utils.to_categorical(Y,10)
  return X_p, Y_p
  

In [4]:
# load and split data
# The data, split between train and test sets:
# (x_train, y_train), (x_test, y_test) = K.datasets.cifar100.load_data()
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
img_rows, img_cols = 32, 32
print('x_train shape:', x_train.shape)
print('y_train shape:', y_train.shape)
print('x_test shape:', x_test.shape)
print('y_test shape:', y_test.shape)


170498071/170498071 [==============================] - 13s 0us/step
x_train shape: (50000, 32, 32, 3)
y_train shape: (50000, 1)
x_test shape: (10000, 32, 32, 3)
y_test shape: (10000, 1)


In [5]:
# Preprocess data
# Next, we are going to call our function with the parameters loaded from the Fashion Mnist database.

x_train, y_train = preprocess_data(x_train, y_train)
x_test, y_test = preprocess_data(x_test, y_test)
print('x_train shape:', x_train.shape)
print('y_train shape:', y_train.shape)
print('x_test shape:', x_test.shape)
print('y_test shape:', y_test.shape)

x_train shape: (50000, 32, 32, 3)
y_train shape: (50000, 10)
x_test shape: (10000, 32, 32, 3)
y_test shape: (10000, 10)


In [6]:
# Using weights of a trained neural network
# A pretrained model from the Keras Applications has the advantage of allow you to use weights that
# are already calibrated to make predictions. In this case, we use the weights from Imagenet 
# and the network is a ResNet50. The option include_top=False allows feature extraction by removing 
# the last dense layers. This let us control the output and input of the model.

input_t = keras.Input(shape=(32,32,3))
res_model = keras.applications.ResNet50(include_top=False,
                                    weights="imagenet",
                                    input_tensor=input_t)


94765736/94765736 [==============================] - 5s 0us/step


In [7]:
# In this case, we ‘freeze’ all layers except for the last block of the ResNet50.

for layer in res_model.layers[:120]:
  layer.trainable=False

In [8]:
# We can check that we did it correctly with:
# False means that the layer is ‘freezed’ or is not trainable and 
# True that when we run our model, the weights are going to be adjusted.

for i, layer in enumerate(res_model.layers):
  print(i,layer.name,"-",layer.trainable)

0 input_1 - False
1 conv1_pad - False
2 conv1_conv - False
3 conv1_bn - False
4 conv1_relu - False
5 pool1_pad - False
6 pool1_pool - False
7 conv2_block1_1_conv - False
8 conv2_block1_1_bn - False
9 conv2_block1_1_relu - False
10 conv2_block1_2_conv - False
11 conv2_block1_2_bn - False
12 conv2_block1_2_relu - False
13 conv2_block1_0_conv - False
14 conv2_block1_3_conv - False
15 conv2_block1_0_bn - False
16 conv2_block1_3_bn - False
17 conv2_block1_add - False
18 conv2_block1_out - False
19 conv2_block2_1_conv - False
20 conv2_block2_1_bn - False
21 conv2_block2_1_relu - False
22 conv2_block2_2_conv - False
23 conv2_block2_2_bn - False
24 conv2_block2_2_relu - False
25 conv2_block2_3_conv - False
26 conv2_block2_3_bn - False
27 conv2_block2_add - False
28 conv2_block2_out - False
29 conv2_block3_1_conv - False
30 conv2_block3_1_bn - False
31 conv2_block3_1_relu - False
32 conv2_block3_2_conv - False
33 conv2_block3_2_bn - False
34 conv2_block3_2_relu - False
35 conv2_block3_3_conv - 

In [9]:
    # Add Flatten and Dense layers on top of Resnet
    # Now, we need to connect our pretrained model with the new layers 
    # of our model. We can use global pooling or a flatten layer to connect 
    # the dimensions of the previous layers with the new layers. 
    
    # to_res = (224, 224)
    to_res = (32, 32)
    model = K.models.Sequential()
    model.add(K.layers.Lambda(lambda image: tf.image.resize(image, to_res))) 
    model.add(res_model)
    model.add(K.layers.Flatten())
    model.add(K.layers.BatchNormalization())
    model.add(K.layers.Dense(256, activation='relu'))
    model.add(K.layers.Dropout(0.5))
    model.add(K.layers.BatchNormalization())
    model.add(K.layers.Dense(128, activation='relu'))
    model.add(K.layers.Dropout(0.5))
    model.add(K.layers.BatchNormalization())
   # model.add(K.layers.Dense(64, activation='relu'))
   # model.add(K.layers.Dropout(0.5))
   # model.add(K.layers.BatchNormalization())
    model.add(K.layers.Dense(10, activation='softmax'))

In [11]:
# Compile model and train
# Results
# We obtained an accuracy of 94% on training set and 90% on validation with 10 epochs.
# In the 8th epoch, the values are very similar and it is interesting to note that 
# in the first validation accuracy is higher than training. 
# This is because of dropout use, which in Keras, it has a different behavior 
# for training and testing. In testing time, all the features are ready and 
# the dropout is turned off, resulting in a better accuracy. 
# This readjust on the last epochs since the model continues changing on the training.

model.compile(loss='categorical_crossentropy',
                  optimizer=K.optimizers.RMSprop(lr=2e-5),
                  metrics=['accuracy'])
history = model.fit(x_train, y_train, batch_size=32, epochs=20, verbose=1,
                        validation_data=(x_test, y_test)
                       )
model.summary()


Epoch 1/20
1563/1563 [==============================] - 67s 39ms/step - loss: 2.0505 - accuracy: 0.2980 - val_loss: 1.7742 - val_accuracy: 0.3608
Epoch 2/20
1563/1563 [==============================] - 60s 39ms/step - loss: 1.9655 - accuracy: 0.3300 - val_loss: 1.7988 - val_accuracy: 0.3861
Epoch 3/20
1563/1563 [==============================] - 61s 39ms/step - loss: 1.8885 - accuracy: 0.3546 - val_loss: 1.7056 - val_accuracy: 0.3982
Epoch 4/20
1563/1563 [==============================] - 61s 39ms/step - loss: 1.8183 - accuracy: 0.3785 - val_loss: 1.6993 - val_accuracy: 0.4216
Epoch 5/20
1563/1563 [==============================] - 60s 38ms/step - loss: 1.7592 - accuracy: 0.4021 - val_loss: 1.5552 - val_accuracy: 0.4650
Epoch 6/20
1563/1563 [==============================] - 59s 38ms/step - loss: 1.7146 - accuracy: 0.4156 - val_loss: 1.5928 - val_accuracy: 0.4607
Epoch 7/20
1563/1563 [==============================] - 60s 38ms/step - loss: 1.6703 - accuracy: 0.4315 - val_loss: 1.5516 -